# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities_csv = "../python-api-challenge/output_data/cities.csv"
cities_weather_df = pd.read_csv(cities_csv)

cities_weather_df.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,5.360000e+02
mean,267.500000,24.419948,21.493891,47.297649,71.703358,53.876866,9.081866,1.612723e+09
std,154.874142,31.959339,86.391972,32.841505,20.702243,41.667304,6.298586,1.444475e+02
min,0.000000,-54.800000,-179.166700,-43.600000,3.000000,0.000000,0.290000,1.612723e+09
25%,133.750000,-1.630950,-50.060325,23.000000,62.000000,1.000000,4.605000,1.612723e+09
50%,267.500000,32.813850,21.811950,57.200000,76.000000,75.000000,7.695000,1.612723e+09
75%,401.250000,51.219025,86.279175,75.995000,87.000000,99.000000,12.497500,1.612723e+09
max,535.000000,78.218600,179.316700,91.890000,100.000000,100.000000,47.180000,1.612723e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# configure api key
gmaps.configure(api_key=g_key)

# store cities
locations = cities_weather_df[["Lat", "Lng"]]

humidity = cities_weather_df["Humidity"].astype(float)

In [10]:
# Plot Heatmap
humid_map = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
humid_map.add_layer(heat_layer)

# Display figure
humid_map

Figure(layout=FigureLayout(height='420px'))

In [11]:
weather_dropna = cities_weather_df.dropna()

weather_dropna.head(20)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stokmarknes,68.5646,14.9108,32.00,59,75,5.75,NO,1612723225
1,1,georgetown,5.4112,100.3354,80.60,83,20,4.03,MY,1612722954
2,2,avarua,-21.2078,-159.7750,80.60,83,20,5.75,CK,1612723225
3,3,mar del plata,43.4002,-4.0094,48.20,76,75,5.75,ES,1612723225
4,4,lorengau,-2.0226,147.2712,81.81,79,100,7.02,PG,1612723226
5,5,busselton,-33.6500,115.3333,64.00,99,98,9.73,AU,1612723226
6,6,port elizabeth,-7.0349,29.7638,74.75,74,100,3.98,CD,1612723226
7,7,longyearbyen,78.2186,15.6401,15.80,73,75,19.57,SJ,1612723168
8,8,albany,42.6001,-73.9662,30.99,74,100,4.23,US,1612722927
9,9,kununurra,-15.7667,128.7333,78.80,83,75,6.04,AU,1612722949


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# A max temperature lower than 80 degrees but higher than 70.
cities_over70 = weather_dropna[weather_dropna["Max Temp"] > 70]
cities_under80 = cities_over70[cities_over70["Max Temp"] < 80]

# Wind speed less than 10 mph.
cities_less10wspeed = cities_under80[cities_under80["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather = cities_less10wspeed[cities_less10wspeed["Cloudiness"] == 0]



ideal_weather.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.0,15.000000,1.500000e+01
mean,253.866667,-2.544433,66.135313,76.240667,67.666667,0.0,6.265333,1.612723e+09
std,152.934097,20.264790,61.112158,2.829663,17.232306,0.0,1.659358,1.408924e+02
min,51.000000,-33.456900,-70.648300,70.720000,31.000000,0.0,3.470000,1.612723e+09
25%,151.000000,-21.348900,55.547400,74.830000,55.500000,0.0,5.315000,1.612723e+09
50%,215.000000,6.124100,72.700000,77.000000,71.000000,0.0,5.750000,1.612723e+09
75%,344.000000,16.998650,98.982400,78.485000,78.000000,0.0,7.460000,1.612723e+09
max,525.000000,20.900000,167.416400,78.800000,94.000000,0.0,9.570000,1.612723e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_map = ideal_weather[["Lat", "Lng"]]

hotel_humid = ideal_weather["Humidity"].astype(float)

# Plot Heatmap
hotel_plot = gmaps.figure()

# Create heat layer
hotel_heat_layer = gmaps.heatmap_layer(hotel_map, weights=hotel_humid, 
                                 dissipating=False, max_intensity=94,
                                 point_radius=2.5)

# Add layer
hotel_plot.add_layer(hotel_heat_layer)

# Display figure
hotel_plot

Figure(layout=FigureLayout(height='420px'))

In [16]:
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(ideal_weather["City"])):

    lat = ideal_weather.iloc[city]["Lat"]
    lng = ideal_weather.iloc[city]["Lng"]

    city_location = f"{lat},{lng}"

    params = {
        "location": city_location, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(query_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
ideal_weather["Hotel Name"] = hotels
ideal_weather

<ipython-input-16-de2d02846831>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather["Hotel Name"] = hotels


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,51,castro,-24.7911,-50.0119,76.24,47,0,3.47,BR,1612723236,CHACARA BAILLY
79,79,karratha,-20.7377,116.8463,78.24,71,0,9.57,AU,1612723242,ibis Styles Karratha
86,86,hambantota,6.1241,81.1185,77.00,94,0,8.05,LK,1612723246,Bungalow 63
143,143,beruwala,6.4788,79.9828,75.70,89,0,4.88,LK,1612723024,The Palms Hotel
159,159,chinchani,19.8667,72.7000,73.15,48,0,8.43,IN,1612723263,Vasantshrushti Farm
209,209,veraval,20.9000,70.3667,70.72,51,0,6.06,IN,1612722803,Lords Inn Somnath
211,211,lakatoro,-16.0999,167.4164,78.73,68,0,6.40,VU,1612723276,Lakatoro Palm Lodge
215,215,saint-joseph,-21.3667,55.6167,78.80,78,0,5.75,RE,1612723277,"""Plantation Bed and Breakfast"
234,234,jizan,17.3333,42.6667,77.00,69,0,4.61,SA,1612723282,شهدان. ابوالحجر
238,238,saint-philippe,-21.3585,55.7679,78.80,78,0,5.75,RE,1612723283,"Chambres d'hôte ""La Trinité"""


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel Name: {hotel}" for hotel in hotels])
hotel_plot.add_layer(markers)

# Display figure
hotel_plot

Figure(layout=FigureLayout(height='420px'))